# EfficientDet

ARXIV [https://arxiv.org/pdf/1911.09070.pdf](https://arxiv.org/pdf/1911.09070.pdf)  
Github [https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch](https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch)  

EfficientDet employs EfficientNet as the backbone network, BiFPN as the feature network, and shared class/box prediction network. Both BiFPN layers and class/box net layers are repeated multiple times based on different resource constraints.

[Object Detection SOTA model](https://paperswithcode.com/sota/object-detection-on-coco)  
This page shows object detection models' score on COCO test-dev. I focus AP50 score, because this competitions' metric is AP50.    
1. DyHead (Based Swin-L) : 78.5
2. DetectoRS (Based ResNeXt) : 74.2
3. YOLOv4-P7 (Based Scaled-YOLO) : 73.3
4. EfficientDet-D7 (Based EfficientNet) : 72.4
5. YOLOv4-608 (Based YOLO) : 65.7

<figure>
<img src="https://blog.roboflow.com/content/images/2020/06/yolov5-performance.png" style="width:700px">
    <figcaption>EfficientDet is better model than YOLOv5 on AP.</figcaption>
</figure>


Everyone used model based YOLOv4 or YOLOv5, but this model isn't SOTA model. I'll try EfficientDet first and then Scaled-YOLOv4, DyHead.  
  
## Three notebooks summarize how to use this model.
#### Preprocessing
https://www.kaggle.com/adldotori/efficientdet-preprocessing-better-than-yolov5/
#### Training
https://www.kaggle.com/adldotori/efficientdet-training-better-than-yolov5/
#### Inference
https://www.kaggle.com/adldotori/efficientdet-inference-better-than-yolov5/
  
  
  
This notebook is second notebook which includes how to training EfficientDet.  
Let's start!  

This picture shows the rough structure of efficientdet.
![image](https://aihub-storage.s3.ap-northeast-2.amazonaws.com/file/efficientdet.png)

# Environment

In [ ]:
!cp -r ../input/efficientdet-preprocessing-better-than-yolov5/Yet-Another-EfficientDet-Pytorch/ ./

In [ ]:
import os
os.chdir('Yet-Another-EfficientDet-Pytorch/')

In [ ]:
!pip install pycocotools webcolors

# Training

## Train a custom dataset with pretrained weights (Highly Recommended)

In [ ]:
!python train.py -c 0 -p siim --head_only True --lr 5e-3 --batch_size 32 --load_weights weights/efficientdet-d0.pth --num_epochs 10 --save_interval 100

## Resume training

In [ ]:
! python train.py -c 0 -p siim --head_only False --lr 1e-3 --batch_size 16 --load_weights last  --num_epochs 40 --save_interval 100

# Evaluation

## mAP

In [ ]:
weight_file = !ls logs/siim/
! python coco_eval.py -c 0 -p siim -w "logs/siim/{weight_file[-2]}"

We focus Average Precision (AP) @[IoU=0.50] score.

## test image result

In [ ]:
import torch
import random
from torch.backends import cudnn

from backbone import EfficientDetBackbone
import cv2
import matplotlib.pyplot as plt
import numpy as np

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess
from colorama import Fore

In [ ]:
compound_coef = 0
force_input_size = None  # set None to use default size
img_path = 'datasets/siim/val/'
test_cnt = 30

threshold = 0.1
iou_threshold = 0.1

use_cuda = True
use_float16 = False
cudnn.fastest = True
cudnn.benchmark = True

class_names = ['Typical', 'Indeterminate', 'Atypical']
label2color = {
    '[0, 0, 0]': [0, 0, 0], # Typical Appearance
    '[1, 0, 0]': [66,9,255], # Typical Appearance
    '[0, 1, 0]': [255,186,8], # Indeterminate Appearance
    '[0, 0, 1]': [247,37,69], # Atypical Appearance
}
name2color = {
    'Negative' : [0, 0, 0], # Typical Appearance
    'Typical': [66,9,255], # Typical Appearance
    'Indeterminate': [255,186,8], # Indeterminate Appearance
    'Atypical': [247,37,69], # Atypical Appearance
}

obj_list = ['Typical','Indeterminate', 'Atypical']

# tf bilinear interpolation is different from any other's, just make do
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size

imagelist = os.listdir(img_path)
random.shuffle(imagelist)
images = imagelist[:test_cnt]

![](https://aihub-storage.s3.ap-northeast-2.amazonaws.com/file/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA_2021-07-18_%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE_10.53.32.png)

## load etc files

In [ ]:
import pandas as pd
train_image = pd.read_csv('datasets/train_image.csv')
import json
with open('datasets/siim/annotations/instances_val.json', 'r') as f:
    files = json.load(f)

## Inference

In [ ]:
imgs = []
for image in images:
    ori_imgs, framed_imgs, framed_metas = preprocess(img_path + image, max_size=input_size)

    if use_cuda:
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
    else:
        x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

    x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

    model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),

                                 # replace this part with your project's anchor config
                                 ratios=[(1.0, 1.0), (1.3, 0.8), (1.9, 0.5)],
                                 scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

    model.load_state_dict(torch.load('logs/siim/'+weight_file[-2]))
    model.requires_grad_(False)
    model.eval()

    if use_cuda:
        model = model.cuda()
    if use_float16:
        model = model.half()

    with torch.no_grad():
        features, regression, classification, anchors = model(x)

        regressBoxes = BBoxTransform()
        clipBoxes = ClipBoxes()

        out = postprocess(x,
                          anchors, regression, classification,
                          regressBoxes, clipBoxes,
                          threshold, iou_threshold)

    out = invert_affine(framed_metas, out)

    for i in range(len(ori_imgs)):
        if len(out[i]['rois']) == 0:
            continue
        for j in range(len(out[i]['rois'])):
            (x1, y1, x2, y2) = out[i]['rois'][j].astype(np.int)
            obj = obj_list[out[i]['class_ids'][j]]
            cv2.rectangle(ori_imgs[i], (x1, y1), (x2, y2), name2color[obj], 2)
            score = float(out[i]['scores'][j])

            cv2.putText(ori_imgs[i], '{:.3f}'.format(score),
                        (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255,255,255), 1)
            
    tmp = train_image[train_image.id==image[:-4]]
    label = tmp['label']
    img = cv2.imread(img_path + image)
    claz = tmp[class_names].values
    color = label2color[str(claz.tolist()[0])]

    for i in files['images']:
        if i['file_name'] == image:
            image_id = i['id']
    bboxes = []
    for i in files['annotations']:
        if i['image_id'] == image_id:
            bboxes.append(i['bbox'])
            
    for box in bboxes:
        img = cv2.rectangle(
            img,
            (int(box[0]), int(box[1])),
            (int(box[2]+box[0]), int(box[3]+box[1])),
            color, 2
        )
    imgs += [ori_imgs[0], img]

![](https://aihub-storage.s3.ap-northeast-2.amazonaws.com/file/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA_2021-07-18_%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE_10.53.32.png)

In [ ]:
fig = plt.figure(figsize=(2*5, len(imgs)//2*5))
for i, img in enumerate(imgs[:30]):
    img = cv2.resize(img, (300,300))
    ax = fig.add_subplot(len(imgs)//2, 2, i+1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == 0:
        ax.set_title('Prediction Image',fontsize=20)
    elif i == 1:
        ax.set_title('Ground Truth Image',fontsize=20)
    plt.imshow(img)
plt.show()

# Finished!

The white letter means confidence. This result is pretty cool. The model is used to draw boxes by image and classify studies through labels on those boxes. This content will be shared on the next notebook. Please look forward to it.